# PySpark training for data engineers
## 04. Data Filtering

### Goal

Practise with filtering on the two RDDs created in the earlier notebooks. Both RDDs are converted to dataframes and some simple filtering examples are shown.

### Highlights

* `sqlContext.createDataFrame()` creates a Spark dataframe from a RDD.
* If the schema cannot be inferred from the RDD, a schema has to be supplied when converting the RDD to a dataframe.
* Using `dataframe.select()` and `dataframe.where()` data can be selected.
* By using `dataframe.groupBy()` data can be sorted (and counted, etc).

### Implementation

In [1]:
from pyspark import SparkConf, SparkContext
config = SparkConf().setMaster('local')
spark = SparkContext.getOrCreate(conf=config)

#### XML

In [2]:
xmlrdd = spark.pickleFile('xml-pickle-03/')

In [3]:
xmlrdd.collect()

[{'text': 'One', 'info': 'testfile'}, {'text': 'Two', 'info': 'testfile'}]

Let us create a SQLContext to get more flexibility in our Spark environment.

In [4]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)

Now we can convert the RDD to a dataframe which makes it easier to work with the data:

In [5]:
xmldf = sqlContext.createDataFrame(xmlrdd)

/usr/local/spark/python/pyspark/sql/session.py:360: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


In [6]:
xmldf.show()

+--------+----+
|    info|text|
+--------+----+
|testfile| One|
|testfile| Two|
+--------+----+



From the above print we can observe that the columns are correct, both columns (info and text) are inferred from the RDD.

In [7]:
xmldf.where(xmldf['text'] == 'One').show()

+--------+----+
|    info|text|
+--------+----+
|testfile| One|
+--------+----+



In [8]:
xmldf.where(xmldf['text'] == 'Two').show()

+--------+----+
|    info|text|
+--------+----+
|testfile| Two|
+--------+----+



#### CSV

In [9]:
csvrdd = spark.pickleFile('csv-pickle-03/')

In [10]:
csvrdd.collect()

['john,doe,male,32',
 'jake,doe,male,16',
 'jane,doe,female,31',
 'janet,doe,female,13']

First we need to create proper Rows from each CSV line by using a mapping.

In [12]:
from pyspark.sql import Row

def processCSV(row):
    # Split the row into a list
    row = row.split(',')
    # Return the four fields
    return Row(row[0], row[1], row[2], row[3])

csvrdd = csvrdd.map(lambda row: processCSV(row))
csvrdd.collect()

[<Row(john, doe, male, 32)>,
 <Row(jake, doe, male, 16)>,
 <Row(jane, doe, female, 31)>,
 <Row(janet, doe, female, 13)>]

We define the schema of the data so the `createDataFrame` does not have to infer the schema.

In [13]:
from pyspark.sql.types import StructField, StructType, StringType, IntegerType
schema = StructType([
            StructField("first_name", StringType(), True),
            StructField("last_name", StringType(), True),
            StructField("gender", StringType(), True),
            StructField("age", StringType(), True)
        ])

In [14]:
csvdf = sqlContext.createDataFrame(csvrdd, schema=schema)

In [15]:
csvdf.show()

+----------+---------+------+---+
|first_name|last_name|gender|age|
+----------+---------+------+---+
|      john|      doe|  male| 32|
|      jake|      doe|  male| 16|
|      jane|      doe|female| 31|
|     janet|      doe|female| 13|
+----------+---------+------+---+



In [16]:
csvdf.select(csvdf.age > 30).collect()

[Row((age > 30)=True),
 Row((age > 30)=False),
 Row((age > 30)=True),
 Row((age > 30)=False)]

In [17]:
csvdf.select(csvdf.age > 30).show()

+----------+
|(age > 30)|
+----------+
|      true|
|     false|
|      true|
|     false|
+----------+



In [18]:
csvdf.select('first_name', csvdf.age > 30).collect()

[Row(first_name='john', (age > 30)=True),
 Row(first_name='jake', (age > 30)=False),
 Row(first_name='jane', (age > 30)=True),
 Row(first_name='janet', (age > 30)=False)]

In [19]:
from pyspark.sql import functions as psf
csvdf.select('first_name', psf.when(csvdf.age > 30, 1).otherwise(0)).show()

+----------+--------------------------------------+
|first_name|CASE WHEN (age > 30) THEN 1 ELSE 0 END|
+----------+--------------------------------------+
|      john|                                     1|
|      jake|                                     0|
|      jane|                                     1|
|     janet|                                     0|
+----------+--------------------------------------+



In [20]:
csvdf[csvdf.first_name.isin("jane","janet")].show()

+----------+---------+------+---+
|first_name|last_name|gender|age|
+----------+---------+------+---+
|      jane|      doe|female| 31|
|     janet|      doe|female| 13|
+----------+---------+------+---+



In [21]:
csvdf[csvdf.first_name.startswith("ja")].show()

+----------+---------+------+---+
|first_name|last_name|gender|age|
+----------+---------+------+---+
|      jake|      doe|  male| 16|
|      jane|      doe|female| 31|
|     janet|      doe|female| 13|
+----------+---------+------+---+



Count the different genders:

In [27]:
csvdf.groupBy('gender').count().show()

+------+-----+
|gender|count|
+------+-----+
|female|    2|
|  male|    2|
+------+-----+



Sort by age:

In [29]:
csvdf.sort('age').show()

+----------+---------+------+---+
|first_name|last_name|gender|age|
+----------+---------+------+---+
|     janet|      doe|female| 13|
|      jake|      doe|  male| 16|
|      jane|      doe|female| 31|
|      john|      doe|  male| 32|
+----------+---------+------+---+



### Save the dataframe to Parquet

In [24]:
csvdf.write.parquet('notebook-04-parquet')

In [25]:
ls notebook-04-parquet/

part-00000-9825b615-2bf6-4cdc-862a-3a875977eaf4-c000.snappy.parquet  _SUCCESS
